# Getting the page

In [79]:
from selenium import webdriver
import pandas as pd
from pandas import ExcelWriter

from openpyxl.workbook import Workbook

pd.set_option('display.max_rows', 10, 'display.max_columns',
              None, 'display.width', None)

browser = webdriver.Firefox()

browser.get(
    'https://www.mismarcadores.com/futbol/chile/primera-division/resultados/')


## Selecting information

In [81]:
aux_ids = browser.find_elements_by_css_selector(
    '.event__match.event__match--static.event__match--oneLine')
partidos = []
for aux in aux_ids:
    partidos.append([aux.find_elements_by_css_selector('.event__participant.event__participant')[0].text,
                     aux.find_element_by_css_selector(
                         '.event__scores.fontBold').text.replace('\n', ''),
                     aux.find_elements_by_css_selector(
                         '.event__participant.event__participant')[1].text,
                     aux.get_attribute('id')])


## Renaming and ordering data

In [173]:
df = pd.DataFrame(partidos)


In [174]:
df.rename(columns={0: 'Casa', 1: 'R 90 min', 2: 'Visitante', 3: 'uid', 4: 'rc1', 5: 'rc2', 6: 'rc3', 7: 'rc4', 8: 'rc5',
                   9: '', 10: 'rv1', 11: 'rv2', 12: 'rv3', 13: 'rv4', 14: 'rv5',
                   15: '', 16: 'lc1', 17: 'lc2', 18: 'lc3', 19: 'lc4', 20: 'lc5',
                   21: '', 22: 'lv1', 23: 'lv2', 24: 'lv3', 25: 'lv4', 26: 'lv5',
                   }, inplace=True)
df[:5] 


,Casa,R 90 min,Visitante,uid
0,Deportes Iquique,1 - 0,O'Higgins,g_1_WdnnUuIa
1,Antofagasta,0 - 0,Curicó Unido,g_1_MouaRw2I
2,La Serena,1 - 2,Colo Colo,g_1_h6mrVL2g
3,U. De Concepción,2 - 2,Cobresal,g_1_vNkfSJnC
4,U. De Chile,0 - 0,Everton,g_1_AsbwW1mm


## Cleaning uid column

In [175]:
df['uid'] = df['uid'].apply(lambda id: id.replace('g_1_','')[:])
df[:5]

,Casa,R 90 min,Visitante,uid
0,Deportes Iquique,1 - 0,O'Higgins,WdnnUuIa
1,Antofagasta,0 - 0,Curicó Unido,MouaRw2I
2,La Serena,1 - 2,Colo Colo,h6mrVL2g
3,U. De Concepción,2 - 2,Cobresal,vNkfSJnC
4,U. De Chile,0 - 0,Everton,AsbwW1mm


## Custom links

In [183]:
url_aux = 'https://www.mismarcadores.com/partido/'
df['link'] = url_aux + df['uid'] + '/#h2h;overall'

## Export to xlsx file

In [186]:
writer = ExcelWriter('marcadores.xlsx')
df.to_excel(writer, 'Hoja1')
writer.save()

## See the results

In [210]:
clean_data_results = pd.read_excel('marcadores.xlsx')
clean_data_results[:5] 

,Unnamed: 0,Casa,R 90 min,Visitante,uid,link
0,0,Deportes Iquique,1 - 0,O'Higgins,WdnnUuIa,https://www.mismarcadores.com/partido/WdnnUuIa...
1,1,Antofagasta,0 - 0,Curicó Unido,MouaRw2I,https://www.mismarcadores.com/partido/MouaRw2I...
2,2,La Serena,1 - 2,Colo Colo,h6mrVL2g,https://www.mismarcadores.com/partido/h6mrVL2g...
3,3,U. De Concepción,2 - 2,Cobresal,vNkfSJnC,https://www.mismarcadores.com/partido/vNkfSJnC...
4,4,U. De Chile,0 - 0,Everton,AsbwW1mm,https://www.mismarcadores.com/partido/AsbwW1mm...


## Filtering links from the first 5

In [217]:
clean_data_links = clean_data_results['link'][:5]
clean_data_links

for link in clean_data_links[:5]:
    print(link)


https://www.mismarcadores.com/partido/WdnnUuIa/#h2h;overall
https://www.mismarcadores.com/partido/MouaRw2I/#h2h;overall
https://www.mismarcadores.com/partido/h6mrVL2g/#h2h;overall
https://www.mismarcadores.com/partido/vNkfSJnC/#h2h;overall
https://www.mismarcadores.com/partido/AsbwW1mm/#h2h;overall


In [223]:
import requests

response = requests.get('https://www.mismarcadores.com/partido/WdnnUuIa/#h2h;overall')

response.headers['date']


'Mon, 09 Mar 2020 21:01:02 GMT'

## Results filter

In [209]:
clean_data_results.loc[clean_data_results['R 90 min'] == '1 - 0']

,Unnamed: 0,Casa,R 90 min,Visitante,uid,link
0,0,Deportes Iquique,1 - 0,O'Higgins,WdnnUuIa,https://www.mismarcadores.com/partido/WdnnUuIa...
6,6,Curicó Unido,1 - 0,Huachipato,0G2SDKXI,https://www.mismarcadores.com/partido/0G2SDKXI...
15,15,Curicó Unido,1 - 0,Colo Colo,YsUYXTJs,https://www.mismarcadores.com/partido/YsUYXTJs...
20,20,Huachipato,1 - 0,Cobresal,G6JwXmZm,https://www.mismarcadores.com/partido/G6JwXmZm...
31,31,Everton,1 - 0,Coquimbo,xv5zxwLF,https://www.mismarcadores.com/partido/xv5zxwLF...
36,36,Curicó Unido,1 - 0,Deportes Iquique,E5JuGebd,https://www.mismarcadores.com/partido/E5JuGebd...
44,44,Palestino,1 - 0,Huachipato,fempRL6c,https://www.mismarcadores.com/partido/fempRL6c...
47,47,U. La Calera,1 - 0,U. Española,vm5c3IEj,https://www.mismarcadores.com/partido/vm5c3IEj...
50,50,Curicó Unido,1 - 0,La Serena,U5Ft8trE,https://www.mismarcadores.com/partido/U5Ft8trE...
